<a href="https://colab.research.google.com/github/arkabyo/shopifyInventoryUpdateBySKU/blob/main/Shopify_Inventory_Update_using_SKU_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Shopify Bulk Inventory Update Tool (SKU-Based)-Google Colab Version

This notebook provides a powerful tool for updating your Shopify inventory based on SKU (Stock Keeping Unit) information from a new inventory file. Shopify does not handle direct inventory updates using SKUs through its interface; this notebook bridges that gap, allowing you to easily update inventory quantities for specific SKUs.

## How It Works

1. **Export your current Shopify Inventory:** Before running this notebook, export your current Shopify Inventory by navigating to: **Shopify Admin > Products > Inventory**. Then, **Choose Location** and Apply appropriate **filters** and then click **Export**. In the Export window, for Location, please select **One Location** at a time and for Quantity type, choose **All quantity types**.
2. **Upload your Shopify Inventory Export File**.
3. **Upload your New Inventory File** that includes the updated quantities.
4. **Process and Export the Inventory**: Click the "Export Inventory" button to update your inventory based on the new data. A downloadable file will be automatically generated for you to import back into Shopify.

## Step-by-Step Instructions

Please follow the detailed instructions within this notebook to ensure successful uploading of your inventory files and the export of your updated inventory. This tool is designed with a focus on simplicity and efficiency, facilitating a smooth bulk inventory update process based on SKU information.

## Requirements

- This notebook is intended for use in Google Colab.
- Ensure your inventory files are in CSV format before uploading.
- To use this script on your local machine, please use the file `Shopify_Inventory_Update_using_SKU_Local_Machine.ipynb` in this repository.


In [ ]:
# Import necessary libraries
from google.colab import files
import pandas as pd
import io

# Function to upload a file and return its DataFrame
def upload_file(description):
    print(f"{description}")
    uploaded = files.upload()  # This will prompt for file upload
    if uploaded:
        file_name = next(iter(uploaded))
        print(f"File '{file_name}' uploaded successfully.")
        return pd.read_csv(io.BytesIO(uploaded[file_name]))
    else:
        print("\nNo file uploaded.")
        return None

# Upload Shopify Inventory Export File
shopify_file = upload_file("Upload Shopify Inventory File")

# Upload New Inventory File
new_inventory = upload_file("\nUpload New Inventory File")

if shopify_file is not None and new_inventory is not None:
    print("\nProcessing... Please wait.")

    # Process the files
    # Convert SKU columns to uppercase for case-insensitive comparison
    shopify_file['SKU'] = shopify_file['SKU'].str.upper()
    new_inventory['SKU'] = new_inventory['SKU'].str.upper()

    # Replace "ABC" with "XYZ" in the SKU columns to match both the SKU Files (if needed)
    shopify_file['SKU'] = shopify_file['SKU'].str.replace("ABC", "XYZ") #C hange ABC & XYZ as needed
    new_inventory['SKU'] = new_inventory['SKU'].str.replace("ABC", "XYZ") #C hange ABC & XYZ as needed

    # Set initial Shopify values for "Available" and "On Hand" to 0
    shopify_file['Available'] = 0
    shopify_file['On hand'] = 0

    # Compare and update data
    for sku in shopify_file['SKU']:
        if sku in new_inventory['SKU'].values:
            quantity = new_inventory.loc[new_inventory['SKU'] == sku, 'Quantity'].values[0]
            shopify_file.loc[shopify_file['SKU'] == sku, 'Available'] = quantity
            shopify_file.loc[shopify_file['SKU'] == sku, 'On hand'] = quantity

    # Export the updated Shopify inventory to a CSV file
    output_filename = 'UpdatedShopifyInventory.csv'
    shopify_file.to_csv(output_filename, index=False)

    print("\nProcessing completed. The file is now being downloaded...")
    files.download(output_filename)
else:
    print("\nPlease make sure both files are uploaded.")
